##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial sobre bandidos multibrazo con características por brazo

### Introducción

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/per_arm_bandits_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/agents/tutorials/per_arm_bandits_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/agents/tutorials/per_arm_bandits_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

Este tutorial es una guía paso a paso del uso de la biblioteca TF-Agents para problemas de bandidos contextuales donde las acciones (brazos) tienen sus propias características, como una lista de películas representadas por características (género, año de estreno, ...).

### Requisito previo

Se asume que el lector tiene cierto grado de familiaridad con la biblioteca Bandit de TF-Agents, en particular, que ha trabajado con el [tutorial para Bandits en TF-Agents](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb) antes de leer este tutorial.

## Bandidos multibrazo con características de brazo

En el "clásico" contexto de bandidos multibrazo, un agente recibe un vector de contexto (también conocido como observación) en cada paso de tiempo y tiene que elegir entre un conjunto finito de acciones numeradas (brazos) para maximizar la recompensa acumulada.

Supongamos que un agente recomienda a un usuario la próxima película que debe ver. Cada vez que tiene que tomar una decisión, el agente recibe como contexto cierta información sobre el usuario (historial de películas vistas, género preferido, etc.), así como una lista de películas entre las que se puede elegir.

Podríamos tratar de plantear este problema usando la información del usuario como contexto y los brazos serían `movie_1, movie_2, ..., movie_K`, pero este enfoque presenta varias limitaciones:

- El número de acciones debería ser todas las películas del sistema y resulta complicado agregar una nueva película.
- El agente debe aprender un modelo para cada película.
- No se tiene en cuenta la similitud entre películas.

En lugar de enumerar las películas, podemos probar un enfoque más intuitivo: podemos representar las películas con un conjunto de características, incluido el género, la duración, el reparto, la calificación, el año, etc. Este enfoque tiene múltiples ventajas:

- Generalización entre películas.
- El agente aprende solo una función de recompensa que modela la recompensa con características de usuario y de película.
- Es fácil eliminar o introducir nuevas películas en el sistema.

En esta nueva configuración, ni siquiera es necesario que el número de acciones sea el mismo en cada paso de tiempo.


## Bandidos por brazo en TF-Agents

El paquete Bandit de TF-Agents se desarrolló para que también se pueda usar en el caso por brazo. Hay entornos por brazo y, además, la mayoría de las políticas y agentes pueden operar en modo por brazo.

Antes de meternos de lleno en la codificación de un ejemplo, tenemos que hacer las importaciones necesarias.

### Instalación

In [ ]:
!pip install tf-agents

### Importaciones

In [ ]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts

nest = tf.nest

### Parámetros (siéntase libre de experimentar)

In [ ]:
# The dimension of the global features.
GLOBAL_DIM = 40  #@param {type:"integer"}
# The elements of the global feature will be integers in [-GLOBAL_BOUND, GLOBAL_BOUND).
GLOBAL_BOUND = 10  #@param {type:"integer"}
# The dimension of the per-arm features.
PER_ARM_DIM = 50  #@param {type:"integer"}
# The elements of the PER-ARM feature will be integers in [-PER_ARM_BOUND, PER_ARM_BOUND).
PER_ARM_BOUND = 6  #@param {type:"integer"}
# The variance of the Gaussian distribution that generates the rewards.
VARIANCE = 100.0  #@param {type: "number"}
# The elements of the linear reward parameter will be integers in [-PARAM_BOUND, PARAM_BOUND).
PARAM_BOUND = 10  #@param {type: "integer"}

NUM_ACTIONS = 70  #@param {type:"integer"}
BATCH_SIZE = 20  #@param {type:"integer"}

# Parameter for linear reward function acting on the
# concatenation of global and per-arm features.
reward_param = list(np.random.randint(
      -PARAM_BOUND, PARAM_BOUND, [GLOBAL_DIM + PER_ARM_DIM]))

### Un entorno por brazo simple

El entorno estocástico estacionario, que se explica en el otro [tutorial](https://github.com/tensorflow/agents/tree/master/docs/tutorials/bandits_tutorial.ipynb), tiene una contraparte por brazo.

Para inicializar el entorno por brazo, debemos definir funciones que generen lo siguiente:

- *características globales y por brazo*: estas funciones no tienen parámetros de entrada y, cuando se las llama, generan un solo vector de características (globales o por brazo).
- *recompensas*: esta función toma como parámetro la concatenación de un vector de características globales o por brazo, y luego genera una recompensa. Básicamente, esta es la función que deberá "adivinar" el agente. vale la pena aclarar que en el caso por brazo la función de recompensa es idéntica para cada brazo. Esta es la diferencia fundamental con el caso de bandidos clásico, donde el agente debe calcular las funciones de recompensa de forma independiente para cada brazo.


In [ ]:
def global_context_sampling_fn():
  """This function generates a single global observation vector."""
  return np.random.randint(
      -GLOBAL_BOUND, GLOBAL_BOUND, [GLOBAL_DIM]).astype(np.float32)

def per_arm_context_sampling_fn():
  """"This function generates a single per-arm observation vector."""
  return np.random.randint(
      -PER_ARM_BOUND, PER_ARM_BOUND, [PER_ARM_DIM]).astype(np.float32)

def linear_normal_reward_fn(x):
  """This function generates a reward from the concatenated global and per-arm observations."""
  mu = np.dot(x, reward_param)
  return np.random.normal(mu, VARIANCE)

Ahora ya tenemos todo lo necesario para inicializar nuestro entorno.

In [ ]:
per_arm_py_env = p_a_env.StationaryStochasticPerArmPyEnvironment(
    global_context_sampling_fn,
    per_arm_context_sampling_fn,
    NUM_ACTIONS,
    linear_normal_reward_fn,
    batch_size=BATCH_SIZE
)
per_arm_tf_env = tf_py_environment.TFPyEnvironment(per_arm_py_env)

A continuación, podemos comprobar lo que produce este entorno.

In [ ]:
print('observation spec: ', per_arm_tf_env.observation_spec())
print('\nAn observation: ', per_arm_tf_env.reset().observation)

action = tf.zeros(BATCH_SIZE, dtype=tf.int32)
time_step = per_arm_tf_env.step(action)
print('\nRewards after taking an action: ', time_step.reward)

Vemos que la especificación de observación es un diccionario con dos elementos:

- Uno con la clave `'global'`: esta es la parte del contexto global, con una forma que coincide con el parámetro `GLOBAL_DIM`.
- Uno con la clave `'per_arm'`: este es el contexto por brazo y tiene la forma `[NUM_ACTIONS, PER_ARM_DIM]`. Esta parte es el marcador de posición para las características del brazo para cada brazo en un paso de tiempo.


### El agente LinUCB

El agente LinUCB implementa el algoritmo Bandit, cuyo nombre es idéntico, que calcula el parámetro de la función de recompensa lineal al mismo tiempo que mantiene un elipsoide de confianza en torno al cálculo. El agente elige el brazo con la mayor recompensa esperada, suponiendo que el parámetro se encuentra dentro del elipsoide de confianza.

Para crear un agente se precisa el conocimiento de la observación y la especificación de la acción. A la hora de definir el agente, establecemos el parámetro booleano `accepts_per_arm_features` en `True`.

In [ ]:
observation_spec = per_arm_tf_env.observation_spec()
time_step_spec = ts.time_step_spec(observation_spec)
action_spec = tensor_spec.BoundedTensorSpec(
    dtype=tf.int32, shape=(), minimum=0, maximum=NUM_ACTIONS - 1)

agent = lin_ucb_agent.LinearUCBAgent(time_step_spec=time_step_spec,
                                     action_spec=action_spec,
                                     accepts_per_arm_features=True)

### El flujo de los datos de entrenamiento

Esta sección ofrece un vistazo a la mecánica de cómo las características por brazo pasan de la política al entrenamiento. No dude en pasar a la siguiente sección (Definición de la métrica de arrepentimiento) y volver aquí después si está interesado.

Primero, veamos la especificación de datos en el agente. El atributo `training_data_spec` del agente especifica qué elementos y estructura deben tener los datos de entrenamiento.

In [ ]:
print('training data spec: ', agent.training_data_spec)

Si miramos más de cerca la porción `observation` de la especificación, vemos que no contiene características por brazo.

In [ ]:
print('observation spec in training: ', agent.training_data_spec.observation)

¿Qué pasó con las características por brazo? Para responder a esta pregunta, primero debemos tener en cuenta que, cuando se entrena al agente LinUCB, no necesita características por brazo de **todos** los brazos, solo necesita las del brazo **elegido**. Por lo tanto, tiene sentido eliminar el tensor de forma `[BATCH_SIZE, NUM_ACTIONS, PER_ARM_DIM]`, ya que es implica gran desperdicio, especialmente si el número de acciones es grande.

Pero, de todos modos, ¡las características por brazo del brazo elegido deben estar en alguna parte! Es por eso que nos aseguramos de que la política de LinUCB almacene las características del brazo elegido dentro del campo `policy_info` de los datos de entrenamiento:

In [ ]:
print('chosen arm features: ', agent.training_data_spec.policy_info.chosen_arm_features)

A partir de la forma podemos apreciar que el campo `chosen_arm_features` solo tiene el vector de características de un brazo, y ese será el brazo elegido. Tenga en cuenta que la `policy_info`, y con ella las `chosen_arm_features`, es parte de los datos de entrenamiento, tal y como vimos al inspeccionar la especificación de datos de entrenamiento, y por lo tanto está disponible en el momento del entrenamiento.

### Definición de la métrica de arrepentimiento

Antes de iniciar el bucle de entrenamiento, definimos algunas funciones de utilidad que ayudan a calcular el arrepentimiento de nuestro agente. Estas funciones nos permiten determinar la recompensa óptima esperada teniendo en cuenta el conjunto de acciones (en función de las características de sus brazos) y el parámetro lineal oculto para el agente.

In [ ]:
def _all_rewards(observation, hidden_param):
  """Outputs rewards for all actions, given an observation."""
  hidden_param = tf.cast(hidden_param, dtype=tf.float32)
  global_obs = observation['global']
  per_arm_obs = observation['per_arm']
  num_actions = tf.shape(per_arm_obs)[1]
  tiled_global = tf.tile(
      tf.expand_dims(global_obs, axis=1), [1, num_actions, 1])
  concatenated = tf.concat([tiled_global, per_arm_obs], axis=-1)
  rewards = tf.linalg.matvec(concatenated, hidden_param)
  return rewards

def optimal_reward(observation):
  """Outputs the maximum expected reward for every element in the batch."""
  return tf.reduce_max(_all_rewards(observation, reward_param), axis=1)

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward)

Ya estamos listos para iniciar nuestro bucle de entrenamiento con bandidos. El controlador que se muestra a continuación se ocupa de elegir las acciones mediante el uso de la política, almacenar las acciones elegidas en el búfer de repetición, calcular la métrica de arrepentimiento predefinida y ejecutar el paso de entrenamiento del agente.

In [ ]:
num_iterations = 20 # @param
steps_per_loop = 1 # @param

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.policy.trajectory_spec,
    batch_size=BATCH_SIZE,
    max_length=steps_per_loop)

observers = [replay_buffer.add_batch, regret_metric]

driver = dynamic_step_driver.DynamicStepDriver(
    env=per_arm_tf_env,
    policy=agent.collect_policy,
    num_steps=steps_per_loop * BATCH_SIZE,
    observers=observers)

regret_values = []

for _ in range(num_iterations):
  driver.run()
  loss_info = agent.train(replay_buffer.gather_all())
  replay_buffer.clear()
  regret_values.append(regret_metric.result())


Ahora, veamos el resultado. Si hicimos todo bien, el agente puede calcular bien la función de recompensa lineal y, por lo tanto, la política puede elegir acciones cuya recompensa esperada sea cercana a la óptima. Esto se indica mediante la métrica de arrepentimiento definida anteriormente, que desciende y se acerca a cero.

In [ ]:
plt.plot(regret_values)
plt.title('Regret of LinUCB on the Linear per-arm environment')
plt.xlabel('Number of Iterations')
_ = plt.ylabel('Average Regret')

### Siguientes pasos

El ejemplo anterior se [implementa](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/examples/v2/train_eval_per_arm_stationary_linear.py) en nuestro código base, donde también puede elegir entre otros agentes, incluido el [agente neuronal épsilon-greedy](https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/agents/neural_epsilon_greedy_agent.py).